Set Up

In [3]:
# Install the required libraries
#SAM
!pip install git+https://github.com/facebookresearch/segment-anything.git
#Transformers
!pip install -q git+https://github.com/huggingface/transformers.git
#Datasets to prepare data and monai if you want to use special loss functions
!pip install datasets
!pip install -q monai
#Patchify to divide large images into smaller patches for training. (Not necessary for smaller images)
!pip install patchify

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-l1g0sqbp
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-l1g0sqbp
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=48cf2bea8e8f0c256b6404dd2ec81fca1c3588028047314852ce92ae20e20060
  Stored in directory: /tmp/pip-ephem-wheel-cache-y_9wmb5w/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 

In [4]:
pip install tifffile

In [15]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import os
from patchify import patchify  #Only to handle large images
from torch.utils.data import DataLoader, Dataset
import random
from scipy import ndimage
from google.colab import drive
import tarfile
from torchvision import transforms
from PIL import Image
from skimage import io
from sklearn.metrics import jaccard_score, f1_score

Load data

In [9]:
#connect to the shared drive from the professor
drive.mount('/content/drive')

def extract_tar_gz_files(source_folder, target_folder, file_name):
    os.makedirs(target_folder, exist_ok=True)

    tar_path = os.path.join(source_folder, file_name)

    with tarfile.open(tar_path, 'r:gz') as tar:
        tar.extractall(path=target_folder)
        print(f'Extracted {file_name} in {target_folder}')

source_folder = '/content/drive/MyDrive/sidewalks'

#creating separate target folders for the training, validation, and labels
target_train_folder = '/content/drive/MyDrive/sidewalks_dataset/train'
target_val_folder = '/content/drive/MyDrive/sidewalks_dataset/val'
target_label_folder = '/content/drive/MyDrive/sidewalks_dataset/label'

file_info = {
    'train.tar.gz': target_train_folder,
    'val.tar.gz': target_val_folder,
    'label.tar.gz': target_label_folder
}

#extraction for each file
for file_name, target_folder in file_info.items():
    extract_tar_gz_files(source_folder, target_folder, file_name)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#load tiff stack images and masks

image_file_name = 'sidewalk_train_01.tif'
mask_file_name = 'sidewalk_train_mask_01.tif'

train_images_path = "/content/drive/MyDrive/sidewalks_dataset/train/" + image_file_name
train_masks_path = "/content/drive/MyDrive/sidewalks_dataset/label/" + mask_file_name

#load the images and masks using tifffile
large_images = tifffile.imread(train_image_path)
large_masks = tifffile.imread(train_mask_path)

print(f"Loaded image shape: {large_images.shape}")
print(f"Loaded mask shape: {large_masks.shape}")

Main Code

In [16]:
class SidewalkDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform = None):
        #directories
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        #list all files
        self.images = [os.path.join(image_dir, x) for x in os.listdir(image_dir)]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):

        #file path for image at index
        image_path = self.images[idx]
        #switch to mask directory
        mask_path = image_path.replace(self.image_dir, self.mask_dir)

        image = io.imread(image_path)
        #mask file with greyscale
        mask = io.imread(mask_path, as_gray=True)

        sample = {'image': Image.fromarray(image), 'mask': Image.fromarray(mask)}

        #transformation
        if self.transform:
            sample = self.transform(sample)

        return sample

#transformations
class ToTensor(object):
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']

        #convert to tensor
        image = transforms.functional.to_tensor(image)
        mask = transforms.functional.to_tensor(mask)

        return {'image': image, 'mask': mask}

#define the SAM model (simplified version)
class SAMModel(torch.nn.Module):
    def __init__(self):
        #superclass
        super(SAMModel, self).__init__()

        #encoder
        self.encoder = torch.nn.Sequential(
            #convolutional layer
            torch.nn.Conv2d(3, 64, 3, padding=1),
            #ReLu
            torch.nn.ReLU(),
            #Max pooling
            torch.nn.MaxPool2d(2)
        )

        #decoder
        self.decoder = torch.nn.Sequential(
            #convolutional layer
            torch.nn.Conv2d(64, 1, 3, padding=1),
            #sigmoidal function
            torch.nn.Sigmoid()
        )

    #forward pass
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)

        return x

#metric Function
def calculate_iou(pred, target):

    #calculating the Intersection Over Union score
    intersection = np.logical_and(target, pred)
    union = np.logical_or(target, pred)
    iou_score = np.sum(intersection) / np.sum(union)

    return iou_score

def calculate_dice(pred, target):

    #calculating the dice score
    intersection = np.logical_and(target, pred)
    dice_score = 2 * intersection.sum() / (pred.sum() + target.sum())
    return dice_score

#evaluation Function
def evaluate_model(model, loader, device):
    #evaluation mode
    model.eval()
    total_iou, total_dice, total_f1 = 0, 0, 0

    #disabling the gradient computation(helps reduce memory and speeds it up)
    with torch.no_grad():
        for data in loader:
            inputs, labels = data['image'].to(device), data['mask'].to(device)
            #model predictions
            outputs = model(inputs)
            outputs = outputs.cpu().numpy().round()
            labels = labels.cpu().numpy()

            for pred, label in zip(outputs, labels):
                #flatten
                pred = pred.flatten()
                label = label.flatten()
                #Add up Intersection Over Union Score
                total_iou += calculate_iou(pred, label)
                #Add up Dice Score
                total_dice += calculate_dice(pred, label)
                #Add up the fl score
                total_f1 += f1_score(label, pred, zero_division=1)

    #averages
    avg_iou = total_iou / len(loader)
    avg_dice = total_dice / len(loader)
    avg_f1 = total_f1 / len(loader)

    return avg_iou, avg_dice, avg_f1

#training Function
def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for data in loader:
        #load data and move to device
        inputs, labels = data['image'].to(device), data['mask'].to(device)
        #reset the gradients
        optimizer.zero_grad()
        #forward pass
        outputs = model(inputs)
        #loss computation
        loss = criterion(outputs, labels)
        #backwards pass
        loss.backward()
        optimizer.step()
        #add up the loss
        total_loss += loss.item()

    #average loss
    average_loss = total_loss / len(loader)
    print(f'Average training loss: {average_loss}')

def main():
    #transformations
    transform = transforms.Compose([ToTensor()])

    #Load the dataset with batch size of 4 and shuffling
    dataset = SidewalkDataset(train_images_path, train_masks_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

    #Use GPU for training model like suggested in other notebook
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'Using device: {device}')

    #model
    model = SAMModel().to(device)
    #loss function
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train(model, dataloader, optimizer, criterion, device)
    iou, dice, f1 = evaluate_model(model, dataloader, device)
    print(f'Evaluation Results - IoU: {iou}, Dice: {dice}, F1 Score: {f1}')

    #save the trained model
    model_save_path = '/content/drive/MyDrive/sidewalks_model.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f'Model saved to {model_save_path}')

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_images'